In [1]:
import numpy as np
from _GenerateModels import Unet
from DataGeneratorNew import retina_data1, retina_test
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from _CustomLayers import CustomMeanIoU
import cv2
#from _ERFNETMODEL import ERFNET

In [2]:
x_path = 'train_set/images/*.jpg'
y_path = 'train_set/refined_mask/*.png'
x_path_val = 'val_set/images/*.jpg'
y_path_val = 'val_set/refined_mask/*.png'
x_test_path = 'test_set/images/*.jpg'
y_test_path = 'test_set/refined_mask/*.png'

In [3]:
batch_size = 20
shape = [256, 256]
train_data, val_data = retina_data1(x_path, y_path, x_path_val, y_path_val, center = False, augment = False, shape = shape, batch_size = batch_size)
test_data = retina_test(x_test_path, y_test_path, center = False)

In [4]:
model = Unet.build_model(shape[0], shape[1], 1)

In [5]:
model.compile(optimizer = keras.optimizers.Adam(), loss = keras.losses.BinaryCrossentropy(), 
              metrics = [CustomMeanIoU(2)])

lrs = keras.callbacks.ReduceLROnPlateau(factor = 0.1, patience = 3)
es = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

In [6]:
train_steps = 350//batch_size
val_steps = 100//batch_size

In [7]:
history = model.fit(train_data, validation_data = val_data, epochs = 100, steps_per_epoch = train_steps, 
                    validation_steps = val_steps, callbacks = [lrs, es])

Epoch 1/100
17/17 [==============================] - 27s 526ms/step - loss: 0.2426 - custom_mean_io_u: 0.4912 - val_loss: 0.1785 - val_custom_mean_io_u: 0.4967 - lr: 0.0010
Epoch 2/100
17/17 [==============================] - 12s 467ms/step - loss: 0.0902 - custom_mean_io_u: 0.4967 - val_loss: 0.0761 - val_custom_mean_io_u: 0.4967 - lr: 0.0010
Epoch 3/100
17/17 [==============================] - 8s 456ms/step - loss: 0.0308 - custom_mean_io_u: 0.4967 - val_loss: 0.0373 - val_custom_mean_io_u: 0.4967 - lr: 0.0010
Epoch 4/100
17/17 [==============================] - 8s 450ms/step - loss: 0.0229 - custom_mean_io_u: 0.4968 - val_loss: 0.0294 - val_custom_mean_io_u: 0.4967 - lr: 0.0010
Epoch 5/100
17/17 [==============================] - 8s 448ms/step - loss: 0.0203 - custom_mean_io_u: 0.4967 - val_loss: 0.0290 - val_custom_mean_io_u: 0.4967 - lr: 0.0010
Epoch 6/100
17/17 [==============================] - 8s 452ms/step - loss: 0.0178 - custom_mean_io_u: 0.4967 - val_loss: 0.0235 - val_cust

In [37]:
batch = 0
for img, lab in val_data.take(7):
    batch += 1
    im_no = 0
    preds = model.predict(img)
    preds = tf.where(preds >= 0.5, 255, 0)
    img = tf.cast(img * 255, tf.int32)
    
    for i in range(preds.shape[0]):
        im_no += 1
        save_path = 'results/run1_unet_256x256/' + f'{batch}_{im_no}.png'
        comb_img = cv2.addWeighted(img[i].numpy(), 1, preds[i].numpy(), 0.8, 1)
        cv2.imwrite(save_path, comb_img)
        #plt.imshow(comb_img, 'gray')
        #plt.figure()
        

In [8]:
new_train_data, new_val_data = retina_data1(x_path, y_path, x_path_val, y_path_val, center = False, augment = False, shape = [1024, 1024], batch_size = batch_size)
new_test_data = retina_test(x_test_path, y_test_path, center = False, shape = [1024, 1024])

In [9]:
cmi_train = CustomMeanIoU(2)

In [11]:
for im, labs in new_train_data.take(train_steps):
    
    n_im = tf.image.resize(im, [256, 256], antialias = True)
    n_labs = tf.image.resize(labs, [256, 256], method = 'nearest')
    
    preds = model.predict(n_im)
    
    preds = tf.image.resize(preds, [1024, 1024], method = 'nearest')
    
    cmi_train.update_state(labs, preds)
    
    

In [13]:
cmi_val = CustomMeanIoU(2)

In [14]:
for im, labs in new_val_data.take(val_steps):
    
    n_im = tf.image.resize(im, [256, 256], antialias = True)
    #n_labs = tf.image.resize(labs, [256, 256], method = 'nearest')
    
    preds = model.predict(n_im)
    
    preds = tf.image.resize(preds, [1024, 1024], method = 'nearest')
    
    cmi_val.update_state(labs, preds)

In [15]:
cmi_val.result().numpy()

0.7532597

In [16]:
cmi_test = CustomMeanIoU(2)

In [17]:
for im, labs in new_test_data.take(50//batch_size):
    
    n_im = tf.image.resize(im, [256, 256], antialias = True)
    #n_labs = tf.image.resize(labs, [256, 256], method = 'nearest')
    
    preds = model.predict(n_im)
    
    preds = tf.image.resize(preds, [1024, 1024], method = 'nearest')
    
    cmi_test.update_state(labs, preds)

In [18]:
cmi_test.result().numpy()

0.75073683